In [ ]:
# Markdown 
Otras formas de trabajo
#   Datos a usar
cuántitativas: 
P3094S3 : Cuánto ahorro por cultivar
P3087S1: Valor mensual por practicas o pasantías
P3095S3: Valor ahorra por criar animales
# Cualitativas
P3101: ¿Fue a reuniones familiares durante las ultimas 4 semanas? si o no

# agrupar estas variables para los 3 meses 

In [13]:
library(readr)
library(dplyr)

# Archivos
csv_files <- c(
  "C:/Users/juand/OneDrive/libros/estadistica2taller1/Mayo_2024 1/CSV/Otras formas de trabajo.CSV",
  "C:/Users/juand/OneDrive/libros/estadistica2taller1/Junio_2024/CSV/Otras formas de trabajo.CSV",
  "C:/Users/juand/OneDrive/libros/estadistica2taller1/Julio_2024/CSV/Otras formas de trabajo.CSV"
)

# Columnas necesarias
cols_needed <- c("P3094S3", "P3087S1", "P3095S3", "P3101")

# Función para leer y limpiar
read_and_clean <- function(file) {
  df <- read_delim(
    file,
    delim = ";",
    col_names = TRUE,
    show_col_types = FALSE,
    locale = locale(encoding = "UTF-8"),
    guess_max = 10000
  )
  
  # Seleccionar solo las columnas necesarias
  df <- df %>% select(any_of(cols_needed))
  
  # Limpiar y convertir valores monetarios
  df <- df %>%
    mutate(
      across(c("P3094S3", "P3087S1", "P3095S3"), ~ {
        x <- as.character(.)
        x <- gsub("\\.", "", x)   # eliminar puntos de miles
        x <- gsub(",", ".", x)    # convertir coma a punto decimal si la hay
        suppressWarnings(as.numeric(x))
      })
    )
  
  return(df)
}

# Leer y combinar
combined <- bind_rows(lapply(csv_files, read_and_clean))

# Guardar
write_csv(combined, "C:/Users/juand/OneDrive/libros/estadistica2taller1/Combinado.csv")

cat("✅ Archivo combinado guardado en: C:/Users/juand/OneDrive/libros/estadistica2taller1/Combinado.csv\n")


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


✅ Archivo combinado guardado en: C:/Users/juand/OneDrive/libros/estadistica2taller1/Combinado.csv


In [17]:
# =========================
# Inicio: instalar / cargar paquetes y definir skew/kurt si falta 'moments'
# =========================

required_pkgs <- c("readr","dplyr","ggplot2","boot","rmarkdown","officer","flextable")
to_install <- required_pkgs[!(required_pkgs %in% installed.packages()[,"Package"])]
if(length(to_install)) {
  install.packages(to_install, dependencies = TRUE, repos = "https://cloud.r-project.org")
}
# cargar paquetes
lapply(required_pkgs, function(p) {
  suppressPackageStartupMessages(require(p, character.only = TRUE))
})

# Intentar cargar 'moments' y si no está, definimos funciones manuales
if (!("moments" %in% installed.packages()[, "Package"])) {
  message("Paquete 'moments' no encontrado — usando funciones manuales para skewness/kurtosis.")
  skewness_calc <- function(x, na.rm = TRUE) {
    if (na.rm) x <- x[!is.na(x)]
    n <- length(x)
    if (n < 3) return(NA_real_)
    m <- mean(x)
    s <- sd(x)
    if (s == 0) return(0)
    mean((x - m)^3) / (s^3)
  }
  kurtosis_calc <- function(x, na.rm = TRUE) {
    if (na.rm) x <- x[!is.na(x)]
    n <- length(x)
    if (n < 4) return(NA_real_)
    m <- mean(x)
    s <- sd(x)
    if (s == 0) return(-3) # excess kurtosis = -3 if constant series
    mean((x - m)^4) / (s^4) - 3  # exceso de kurtosis (como moments::kurtosis)
  }
} else {
  library(moments)
  skewness_calc <- function(x, na.rm = TRUE) moments::skewness(x, na.rm = na.rm)
  kurtosis_calc <- function(x, na.rm = TRUE) moments::kurtosis(x, na.rm = na.rm)
}

# =========================
# Continuar con el resto del script (ejemplo: cálculo descriptivos)
# =========================

# Supongamos que ya tienes x_nonmiss definido (vector numérico sin NA):
# Ejemplo:
# x_nonmiss <- na.omit(df$P3095S3)

# Ahora usa las funciones:
# skew_x <- skewness_calc(x_nonmiss)
# kurt_x <- kurtosis_calc(x_nonmiss)



also installing the dependencies 'listenv', 'parallelly', 'rbibutils', 'fontBitstreamVera', 'fontLiberation', 'future', 'globals', 'AsioHeaders', 'V8', 'Rdpack', 'locatexec', 'fontquiver', 'tidyr', 'coda', 'forcats', 'furrr', 'websocket', 'xslt', 'katex', 'minqa', 'nloptr', 'reformulas', 'RcppEigen', 'rvg', 'qpdf', 'devEMF', 'doconv', 'gdtools', 'rsvg', 'data.table', 'bookdown', 'broom', 'broom.mixed', 'chromote', 'equatags', 'lme4', 'officedown', 'pdftools', 'svglite', 'tables', 'webshot2'




package 'listenv' successfully unpacked and MD5 sums checked
package 'parallelly' successfully unpacked and MD5 sums checked
package 'rbibutils' successfully unpacked and MD5 sums checked
package 'fontBitstreamVera' successfully unpacked and MD5 sums checked
package 'fontLiberation' successfully unpacked and MD5 sums checked
package 'future' successfully unpacked and MD5 sums checked
package 'globals' successfully unpacked and MD5 sums checked
package 'AsioHeaders' successfully unpacked and MD5 sums checked
package 'V8' successfully unpacked and MD5 sums checked
package 'Rdpack' successfully unpacked and MD5 sums checked
package 'locatexec' successfully unpacked and MD5 sums checked
package 'fontquiver' successfully unpacked and MD5 sums checked
package 'tidyr' successfully unpacked and MD5 sums checked
package 'coda' successfully unpacked and MD5 sums checked
package 'forcats' successfully unpacked and MD5 sums checked
package 'furrr' successfully unpacked and MD5 sums checked
package

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

Paquete 'moments' no encontrado — usando funciones manuales para skewness/kurtosis.



In [22]:
# ---- Lectura robusta y limpieza de P3095S3 ----
library(readr); library(dplyr); library(ggplot2)

data_path <- "C:/Users/juand/OneDrive/libros/estadistica2taller1/Combinado.csv"
varname <- "P3095S3"
out_dir <- "C:/Users/juand/OneDrive/libros/estadistica2taller1/analisis_P3095S3_check"
dir.create(out_dir, showWarnings = FALSE, recursive = TRUE)

# 1) Leer todo como character para evitar parsing issues
df_char <- readr::read_csv(data_path,
                           col_types = readr::cols(.default = "c"),
                           locale = readr::locale(encoding = "UTF-8"),
                           guess_max = 100000)

# Guardar problemas detectados (si los hay)
probs <- problems(df_char)
if (nrow(probs) > 0) {
  message("Se detectaron problemas de parsing: ", nrow(probs), " filas. Guardando 'parsing_problems.csv'.")
  readr::write_csv(probs, file.path(out_dir, "parsing_problems.csv"))
} else {
  message("No se detectaron problemas de parsing leyendo todo como carácter.")
}

# 2) Si la variable no está exactamente con ese nombre, intentar buscarla (insensitivo a mayúsculas)
if (!varname %in% names(df_char)) {
  guess <- names(df_char)[tolower(names(df_char)) %in% tolower(varname)]
  if (length(guess) == 1) {
    varname_real <- guess
    message("Variable encontrada con nombre alternativo: ", varname_real)
  } else {
    stop("No pude encontrar la variable '", varname, "'. Revisa los nombres de columnas: ", paste(names(df_char)[1:30], collapse = ", "))
  }
} else varname_real <- varname

# 3) Función robusta para convertir textos a numérico (maneja ., , como miles/decimales)
clean_numeric2 <- function(x) {
  x <- as.character(x)
  x[is.na(x)] <- NA_character_
  x <- trimws(x)
  x <- gsub("\\s+", "", x)                # quitar espacios
  x <- gsub("[^0-9\\,\\.-]", "", x)       # eliminar símbolos (moneda, letras)
  # Si el string contiene ambos '.' y ',' asumimos que '.' es separador de miles
  both <- grepl("\\.", x) & grepl(",", x)
  if (any(both)) {
    x[both] <- gsub("\\.", "", x[both])   # quitar puntos de miles
    x[both] <- gsub(",", ".", x[both])    # coma -> punto decimal
  }
  # Si sólo tiene coma, convertir coma -> punto
  only_comma <- grepl(",", x) & !grepl("\\.", x)
  x[only_comma] <- gsub(",", ".", x[only_comma])
  # Si sólo tiene puntos (y parecen decimales), se deja; si eran miles sin coma dec, esto puede fallar.
  # Finalmente convertir a numérico
  suppressWarnings(as.numeric(x))
}

# Convertir la variable
df_char[[varname_real]] <- clean_numeric2(df_char[[varname_real]])
x <- df_char[[varname_real]]
x_nonmiss <- x[!is.na(x)]

# 4) Resumen rápido y diagnóstico
cat("N total filas leídas:", nrow(df_char), "\n")
cat("N no-missing en", varname_real, ":", length(x_nonmiss), "\n")
cat("Valores únicos:", length(unique(x_nonmiss)), "\n")
print(summary(x_nonmiss))

# Top y bottom para inspección
cat("Top 20 valores (mayores):\n")
print(sort(unique(x_nonmiss), decreasing = TRUE)[1:20])
cat("Top 20 valores (menores):\n")
print(sort(unique(x_nonmiss), decreasing = FALSE)[1:20])

# Guardar una muestra (primeras 200 observaciones) para inspección manual en Excel
samp_preview <- df_char %>% select(all_of(varname_real)) %>% slice_head(n = 200)
readr::write_csv(samp_preview, file.path(out_dir, "preview_first200_var.csv"))

# Gráficos simples para ver forma y outliers (png)
png(file.path(out_dir, "hist_var_raw.png"), width = 900, height = 600)
hist(x_nonmiss, breaks = 80, main = paste("Histograma de", varname_real), xlab = varname_real)
dev.off()

png(file.path(out_dir, "boxplot_var_raw.png"), width = 600, height = 400)
boxplot(x_nonmiss, main = paste("Boxplot de", varname_real), ylab = varname_real)
dev.off()

# 5) Detectar outliers extremos (ejemplo: valores > percentile 99.9 o > 10 veces la mediana)
p99.9 <- quantile(x_nonmiss, probs = 0.999, na.rm = TRUE)
med <- median(x_nonmiss, na.rm = TRUE)
extreme_idx <- which(x_nonmiss > p99.9 | x_nonmiss > 10 * med)
cat("Número de observaciones extremas (> P99.9 o > 10*mediana):", length(extreme_idx), "\n")
if (length(extreme_idx) > 0) {
  top_extremes <- sort(unique(x_nonmiss[extreme_idx]), decreasing = TRUE)
  print(head(top_extremes, 20))
  # Guardarlas para inspección
  df_extremes <- df_char %>% mutate(.orig_index = row_number()) %>% filter(!is.na(.data[[varname_real]]) & as.numeric(.data[[varname_real]]) %in% top_extremes)
  readr::write_csv(df_extremes, file.path(out_dir, "extreme_cases_var.csv"))
}

# 6) Recomendar transformación si la variable es monetaria y está muy sesgada
cat("\nSugerencia: variable muy sesgada (mean >> median), se recomienda considerar transformación logarítmica para análisis robustos.\n")
# Crear variable log(1+x) para uso posterior
df_char[[paste0(varname_real, "_log1p")]] <- ifelse(!is.na(df_char[[varname_real]]), log1p(as.numeric(df_char[[varname_real]])), NA_real_)
png(file.path(out_dir, "hist_var_log1p.png"), width = 900, height = 600)
hist(df_char[[paste0(varname_real, "_log1p")]], breaks = 80, main = paste("Histograma log1p(", varname_real, ")"), xlab = paste0("log1p(", varname_real, ")"))
dev.off()

cat("Archivos generados en:", out_dir, "\n")
cat(" - parsing_problems.csv (si existieron problemas)\n")
cat(" - preview_first200_var.csv\n")
cat(" - extreme_cases_var.csv (si hay extremos)\n")
cat(" - hist_var_raw.png, boxplot_var_raw.png, hist_var_log1p.png\n\n")

# FIN del bloque de diagnóstico y limpieza


No se detectaron problemas de parsing leyendo todo como carácter.



N total filas leídas: 166341 
N no-missing en P3095S3 : 4080 
Valores únicos: 152 
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
    1000    30000    60000   159471   150000 20000000 
Top 20 valores (mayores):
 [1] 20000000 10000000  8000000  5000000  4500000  3500000  3000000  2500000
 [9]  2000000  1900000  1800000  1700000  1600000  1500000  1400000  1330000
[17]  1200000  1000000   900000   875000
Top 20 valores (menores):
 [1]  1000  1500  2000  2500  3000  3500  4200  5000  6000  7000  8000  9000
[13]  9999 10000 12000 13000 13500 13600 14000 15000


agg_record_1741283825 
                    2

agg_record_1741283825 
                    2

Número de observaciones extremas (> P99.9 o > 10*mediana): 144 
 [1] 20000000 10000000  8000000  5000000  4500000  3500000  3000000  2500000
 [9]  2000000  1900000  1800000  1700000  1600000  1500000  1400000  1330000
[17]  1200000  1000000   900000   875000

Sugerencia: variable muy sesgada (mean >> median), se recomienda considerar transformación logarítmica para análisis robustos.


agg_record_1741283825 
                    2

Archivos generados en: C:/Users/juand/OneDrive/libros/estadistica2taller1/analisis_P3095S3_check 
 - parsing_problems.csv (si existieron problemas)
 - preview_first200_var.csv
 - extreme_cases_var.csv (si hay extremos)
 - hist_var_raw.png, boxplot_var_raw.png, hist_var_log1p.png



In [21]:
# inspección rápida
summary(x_nonmiss)
length(x_nonmiss)
length(unique(x_nonmiss))
sd(x_nonmiss)
head(x_nonmiss, 20)
problems(df)   # para ver issues de parsing


    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
    1000    30000    60000   159471   150000 20000000 

[1] 4080

[1] 152

[1] 512519

[1]  25000 300000  40000 100000  25000  80000  60000 100000 300000  60000
[11] 190000  70000 150000 200000 150000  70000 100000  50000  20000  60000

row,col,expected,actual,file
<int>,<int>,<chr>,<chr>,<chr>
199,2,1/0/T/F/TRUE/FALSE,1300000,C:/Users/juand/OneDrive/libros/estadistica2taller1/Combinado.csv
662,2,1/0/T/F/TRUE/FALSE,1200000,C:/Users/juand/OneDrive/libros/estadistica2taller1/Combinado.csv
793,2,1/0/T/F/TRUE/FALSE,800000,C:/Users/juand/OneDrive/libros/estadistica2taller1/Combinado.csv
1609,2,1/0/T/F/TRUE/FALSE,80000,C:/Users/juand/OneDrive/libros/estadistica2taller1/Combinado.csv
1617,2,1/0/T/F/TRUE/FALSE,900000,C:/Users/juand/OneDrive/libros/estadistica2taller1/Combinado.csv
2561,2,1/0/T/F/TRUE/FALSE,780000,C:/Users/juand/OneDrive/libros/estadistica2taller1/Combinado.csv
3524,2,1/0/T/F/TRUE/FALSE,200000,C:/Users/juand/OneDrive/libros/estadistica2taller1/Combinado.csv
4169,2,1/0/T/F/TRUE/FALSE,1800000,C:/Users/juand/OneDrive/libros/estadistica2taller1/Combinado.csv
4199,2,1/0/T/F/TRUE/FALSE,960000,C:/Users/juand/OneDrive/libros/estadistica2taller1/Combinado.csv
